<a href="https://colab.research.google.com/github/cruzdany/KERAS-NLP/blob/main/NLP_Assignment05_Maga%C3%B1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import re
from numpy import *  
import matplotlib.pyplot as plt
import numpy as np
import operator
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [8]:
df = pd.read_csv('train.csv',encoding='utf-8',error_bad_lines=False, engine="python")

In [ ]:
"""import nltk
nltk.download('stopwords')"""

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import re

def scrub_words(text):
    # remove html markup
    text=re.sub("(<.*?>)","",text)
    
    #remove non-ascii and digits
    text=re.sub("(\\W|\\d)"," ",text)
  
    #remove whitespace
    text=text.strip()
    return text
  
#Noise removal, stop word removal, normalizing?
def cleanString(s, special_chars = "\":,.@|ðÿœžðÿâœœïÿœžÿºÿÿœžÿ=_"):
    for char in special_chars:
        s = s.replace(char, "")
    s = s.replace("\n", "")
    s = scrub_words(s)
    tokenizer = TweetTokenizer()
    stop_words = set(stopwords.words('english'))
    cleaned_words = [w for w in tokenizer.tokenize(s) if w not in stop_words]
    return " ".join(cleaned_words), tokenizer

def stemWords(sentence):
    stemmer, tokenizer = PorterStemmer(), TweetTokenizer()
    stemmed_words = [stemmer.stem(w) for w in tokenizer.tokenize(sentence)]
    return " ".join(stemmed_words)
    
def cleanFrame(frame):
    frame['clean_tweet'] = frame.comment_text.apply(cleanString)


cleanFrame(df)
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_tweet
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,(ExplanationWhy edits made username Hardcore M...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,(D aww He matches background colour I seemingl...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,(Hey man I really trying edit war It guy const...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,(MoreI make real suggestions improvement I won...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"(You sir hero Any chance remember page, <nltk...."


Import Pickle

In [10]:
import re
special_chars = r"[^0-9!@#\$%\^\&\*_\-']"
def Cleaning(tweet,special_chars):
    tweet = re.sub("\d+", "", tweet)
    tweet=re.sub("h[\S]+//[\S]+", "", tweet)
    tweet=re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", tweet)
    tweet=tweet.lower()
    for char in special_chars:
        tweet = tweet.replace(char, "")    
    tweet=tweet.split(" ")
    return ' '.join(tweet)
Cleaned=[]
for t in df["comment_text"]:
    Cleaned.append(Cleaning(t,special_chars))
df['text_generation'] = Cleaned
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_tweet,text_generation
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,(ExplanationWhy edits made username Hardcore M...,explanationwhy the edits made under my usernam...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,(D aww He matches background colour I seemingl...,daww he matches this background colour im seem...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,(Hey man I really trying edit war It guy const...,hey man im really not trying to edit war its j...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,(MoreI make real suggestions improvement I won...,morei cant make any real suggestions on improv...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"(You sir hero Any chance remember page, <nltk....",you sir are my hero any chance you remember wh...


In [11]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', 'clean_tweet', 'text_generation'],
      dtype='object')

In [12]:
y = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text_generation, y, test_size=0.2,
                                                    random_state=np.random)
print(X_train.shape, X_test.shape, len(y_train), len(y_test))

(127656,) (31915,) 127656 31915


In [14]:
from keras.preprocessing.text import Tokenizer
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
maxlen=400
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
V_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
V_test = pad_sequences(list_tokenized_test, maxlen=maxlen)
print(V_train.shape,V_test.shape)

(127656, 400) (31915, 400)


In [15]:
"""def getModel():
    inp = Input(shape=(maxlen, ))
    embed_size = 128
    x = Embedding(max_features, embed_size)(inp)
    x = LSTM(30, return_sequences=True,name='lstm_layer')(x)
    #x = GlobalMaxPool1D()(x)
    #x = Dropout(0.5)(x)
    x = Dense(20, activation="relu")(x)
    #x = Dropout(0.5)(x)
    x = Dense(6, activation="sigmoid")(x)
    x = 
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model"""

In [32]:
def getModel():
    inp = Input(shape=(maxlen, ))
    embed_size = 128
    x = Embedding(max_features, embed_size)(inp)
    x = LSTM(30, return_sequences=True,name='lstm_layer')(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.5)(x)
    x = Dense(20, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model

'def getModel():\n    inp = Input(shape=(maxlen, ))\n    embed_size = 128\n    x = Embedding(max_features, embed_size)(inp)\n    x = LSTM(30, return_sequences=True,name=\'lstm_layer\')(x)\n    x = GlobalMaxPool1D()(x)\n    x = Dropout(0.5)(x)\n    x = Dense(20, activation="relu")(x)\n    x = Dropout(0.5)(x)\n    x = Dense(6, activation="sigmoid")(x)\n    model = Model(inputs=inp, outputs=x)\n    model.compile(loss=\'binary_crossentropy\',\n                      optimizer=\'adam\',\n                      metrics=[\'accuracy\'])\n    return model'

In [16]:
model = getModel()
batch_size = 32
epochs = 3
file_path="../weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=2)


callbacks_list = [checkpoint, early] #early
model.fit(V_train,y_train,  batch_size=batch_size, epochs=epochs, 
          validation_split=0.1, callbacks=callbacks_list)

model.load_weights(file_path)

Epoch 1/3
3591/3591 [==============================] - ETA: 0s - loss: 0.0975 - accuracy: 0.8830
Epoch 00001: val_loss improved from inf to 0.05595, saving model to ../weights_base.best.hdf5
3591/3591 [==============================] - 795s 221ms/step - loss: 0.0975 - accuracy: 0.8830 - val_loss: 0.0560 - val_accuracy: 0.9945
Epoch 2/3
3591/3591 [==============================] - ETA: 0s - loss: 0.0591 - accuracy: 0.9929
Epoch 00002: val_loss did not improve from 0.05595
3591/3591 [==============================] - 794s 221ms/step - loss: 0.0591 - accuracy: 0.9929 - val_loss: 0.0561 - val_accuracy: 0.9945
Epoch 3/3
3591/3591 [==============================] - ETA: 0s - loss: 0.0535 - accuracy: 0.9934
Epoch 00003: val_loss improved from 0.05595 to 0.05547, saving model to ../weights_base.best.hdf5
3591/3591 [==============================] - 799s 222ms/step - loss: 0.0535 - accuracy: 0.9934 - val_loss: 0.0555 - val_accuracy: 0.9945


In [25]:
from sklearn.pipeline import Pipeline

y_pred = model.predict(V_test)
print(classification_report(y_test,y_pred.round(),digits=6))

              precision    recall  f1-score   support

           0   0.838139  0.716869  0.772776      3041
           1   0.000000  0.000000  0.000000       334
           2   0.789842  0.786171  0.788002      1721
           3   0.000000  0.000000  0.000000        93
           4   0.702811  0.664978  0.683371      1579
           5   0.000000  0.000000  0.000000       281

   micro avg   0.789084  0.650163  0.712919      7049
   macro avg   0.388465  0.361336  0.374025      7049
weighted avg   0.711851  0.650163  0.678849      7049
 samples avg   0.061053  0.058741  0.057187      7049



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
from sklearn.metrics import confusion_matrix

In [21]:
#y_pred = (y_pred > 0.5) 

In [27]:
confusion_matrix(y_test, y_pred)

ValueError: ignored

In [30]:
y_pred

array([[8.65280628e-04, 1.70504572e-12, 6.67351333e-06, 1.05947286e-08,
        9.29132329e-06, 4.00177157e-07],
       [9.02667642e-03, 3.22512017e-09, 2.49862671e-04, 1.60434445e-06,
        3.48836184e-04, 2.40391673e-05],
       [4.60749865e-03, 1.73456582e-10, 9.96369199e-05, 2.07010132e-07,
        1.23351216e-04, 5.44643081e-06],
       ...,
       [5.41269779e-04, 2.33137348e-13, 3.65899541e-06, 2.40138220e-09,
        4.71731073e-06, 1.39313499e-07],
       [9.45121050e-04, 1.11499319e-12, 9.48338311e-06, 6.64421540e-09,
        1.10740630e-05, 3.37052711e-07],
       [4.45145369e-03, 3.06751985e-10, 8.78270221e-05, 3.30830261e-07,
        1.18126605e-04, 6.74480316e-06]], dtype=float32)

In [31]:
y_test

,toxic,severe_toxic,obscene,threat,insult,identity_hate
112778,0,0,0,0,0,0
46170,0,0,0,0,0,0
72363,0,0,0,0,0,0
6521,0,0,0,0,0,0
112502,0,0,0,0,0,0
...,...,...,...,...,...,...
33993,0,0,0,0,0,0
18334,0,0,0,0,0,0
93572,0,0,0,0,0,0
87718,0,0,0,0,0,0


In [28]:
confusion_matrix(y_test.values.argmax(axis=1), y_pred.argmax(axis=1))

array([[31714,     0,     0,     0,     0],
       [  121,     0,     0,     0,     0],
       [    8,     0,     0,     0,     0],
       [   64,     0,     0,     0,     0],
       [    8,     0,     0,     0,     0]])

In [29]:
df.shape

(159571, 10)